In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
plotly.tools.set_credentials_file(username='vvr', api_key='Ker155UU70wk3Ccz6iyM')
import plotly.plotly as py
import plotly.graph_objs as go

color = sns.color_palette("husl", 8)

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


%matplotlib inline

import squarify

import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
#from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
import re


from scipy import sparse


In [2]:
data  = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

In [3]:
data.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,159571,159571,dc236ec3fbbb4c7f,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
comment_text,159571,159571,"C.S Lewis. \n\nSorry, I wasn't trying to use t...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
toxic,159571,NaN,NaN,NaN,0.0958445,0.294379,0,0,0,0,1
severe_toxic,159571,NaN,NaN,NaN,0.00999555,0.0994771,0,0,0,0,1
obscene,159571,NaN,NaN,NaN,0.0529482,0.223931,0,0,0,0,1
threat,159571,NaN,NaN,NaN,0.00299553,0.0546496,0,0,0,0,1
insult,159571,NaN,NaN,NaN,0.0493636,0.216627,0,0,0,0,1
identity_hate,159571,NaN,NaN,NaN,0.00880486,0.0934205,0,0,0,0,1


In [4]:
test.columns

Index(['id', 'comment_text'], dtype='object')

In [5]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
id              153164 non-null object
comment_text    153164 non-null object
dtypes: object(2)
memory usage: 2.3+ MB


In [8]:
data['toxic'].value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [9]:
data['severe_toxic'].value_counts()

0    157976
1      1595
Name: severe_toxic, dtype: int64

In [10]:
data['obscene'].value_counts()

0    151122
1      8449
Name: obscene, dtype: int64

In [11]:
data['threat'].value_counts()

0    159093
1       478
Name: threat, dtype: int64

In [12]:
data['insult'].value_counts()

0    151694
1      7877
Name: insult, dtype: int64

In [13]:
data['identity_hate'].value_counts()

0    158166
1      1405
Name: identity_hate, dtype: int64

In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153164 entries, 0 to 153163
Data columns (total 2 columns):
id              153164 non-null object
comment_text    153164 non-null object
dtypes: object(2)
memory usage: 2.3+ MB


In [15]:
p = pd.DataFrame(0, index=np.arange(len(test.id)), columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

In [16]:
p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153164 entries, 0 to 153163
Data columns (total 6 columns):
toxic            153164 non-null int64
severe_toxic     153164 non-null int64
obscene          153164 non-null int64
threat           153164 non-null int64
insult           153164 non-null int64
identity_hate    153164 non-null int64
dtypes: int64(6)
memory usage: 8.2 MB


In [17]:
test = pd.concat([test, p], axis=1, join='inner')

In [18]:
test.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

# Label encoder

In [19]:
le = LabelEncoder()
le.fit(data['id'])

LabelEncoder()

In [20]:
data['id'] = le.transform(data['id'])

In [21]:
le.fit(test['id'])
le.transform(test['id'])

array([     0,      1,      2, ..., 153161, 153162, 153163], dtype=int64)

# text Processing

1. Remove all non Alphabates

In [22]:
data['count'] = data['comment_text'].apply(lambda x : len(str(x).split(' ')))


In [23]:
test['count'] = test['comment_text'].apply(lambda x : len(str(x).split(' ')))


In [24]:
data['comment_text'] = data['comment_text'].str.replace('[^a-zA-Z]', ' ')

In [25]:
test['comment_text'] = test['comment_text'].str.replace('[^a-zA-Z]', ' ')

In [26]:
from textblob import TextBlob
from textblob import Word
#nltk.download('wordnet')
data['comment_text'].apply(lambda x : ' '.join([Word(word).lemmatize() for word in x.split()]))
data['comment_text'].head()

0    Explanation Why the edits made under my userna...
1    D aww  He matches this background colour I m s...
2    Hey man  I m really not trying to edit war  It...
3      More I can t make any real suggestions on im...
4    You  sir  are my hero  Any chance you remember...
Name: comment_text, dtype: object

In [27]:
test['comment_text'].apply(lambda x : ' '.join([Word(word).lemmatize() for word in x.split()]))
test['comment_text'].head()

0    Yo bitch Ja Rule is more succesful then you ll...
1       From RfC       The title is fine as it is  ...
2            Sources         Zawe Ashton on Lapland...
3     If you have a look back at the source  the in...
4            I don t anonymously edit articles at all 
Name: comment_text, dtype: object

In [28]:
def pir(data):
    t = data.text.str.split(expand=True).stack()
    return t.loc[t.str.len() >= 4].groupby(level=0).apply(' '.join)
data['comment_text'] = data['comment_text'].str.findall('\w{4,}').str.join(' ')
#data.sample()

In [29]:
test['comment_text'] = test['comment_text'].str.findall('\w{4,}').str.join(' ')


In [30]:
stop = stopwords.words('english')
#data['essay'].str.findall('\w{4,}').str.join(' ')
data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#data['essay'].sample()

In [31]:
test['comment_text'] = test['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


In [32]:
freq = pd.Series(' '.join(data['comment_text']).split()).value_counts()


In [33]:
freq_t = pd.Series(' '.join(test['comment_text']).split()).value_counts()

In [34]:
freq2 = freq.head(100)
data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (freq2)]))

In [35]:
freq_t2 = freq_t.head(100)
test['comment_text'] = test['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (freq2)]))

In [36]:
freq3 = freq.tail(175000)
data['comment_text'] = data['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (freq3)]))

In [ ]:
freq_t3 = freq_t.tail(175000)
test['comment_text'] = test['comment_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (freq3)]))

In [ ]:
data['comment_text'].head()

In [ ]:
test['comment_text'].head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(ngram_range = (6,6))
data['comment_text'] =  v.fit_transform(data['comment_text'])
#X_test['essay']  = v.fit_transform(X_test['essay'])

In [ ]:
test['comment_text'] =  v.fit_transform(test['comment_text'])


In [ ]:
X_train = sparse.hstack(data['comment_text'])
x_test = sparse.hstack(test['comment_text'])

# Model Preparation

In [ ]:
'''
y_train = data[['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']].copy()
       '''

In [ ]:
'''
X_train = data.drop(['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'], axis =1)
'''

In [ ]:
#X_test = test[['id', 'comment_text','count']].copy()

In [ ]:
'''
y_test = test[['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']].copy()
'''

1. Logistic Regression

In [2]:
from sklearn.datasets import make_multilabel_classification

# this will generate a random multi-label dataset
X_train, y_train = make_multilabel_classification(sparse = True, n_labels = 20,
return_indicator = 'sparse', allow_unlabeled = False)

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
clf = BinaryRelevance(GaussianNB())

# train
clf.fit(X_train, y_train, 
         )
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
scores = cross_val_score(clf,X_train , y_train, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
clf = BinaryRelevance(LogisticRegressionCV())

# train
clf.fit(X_train, y_train, 
         )
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
scores = cross_val_score(clf,X_train , y_train, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset


# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
clf = ClassifierChain(LogisticRegressionCV())

# train
clf.fit(X_train, y_train, 
         )
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
scores = cross_val_score(clf,X_train , y_train, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset


# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
clf = LabelPowerset(LogisticRegressionCV())

# train
clf.fit(X_train, y_train, 
         )
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
scores = cross_val_score(clf,X_train , y_train, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
from skmultilearn.adapt import MLkNN

clf = MLkNN(k=20)

# train
clf.fit(X_train, y_train)

# predict
predictions = clf.predict(X_test)

#accuracy_score(y_test,predictions)
cv = ShuffleSplit(n_splits=3, test_size=0.3, random_state=0)
scores = cross_val_score(clf,X_train , y_train, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
train.columns  = ['id', 'userid', 'campaignid', 'send_date', 'open', 'click']


In [ ]:
test